In [22]:
import os
if os.getcwd().split('/')[-1] == 'examples':
    os.chdir('..')

import polars as pl
import numpy as np
from polang import polang
from pyparsing import *
from pyparsing import pyparsing_common as ppc

from operator import methodcaller

In [4]:
# Trigonometric functions
expr = polang("alias(sin(a), 'sinus of a')")
print(expr)

df = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2), "b": np.linspace(2,3), "c": np.linspace(3,4)})
df.select([pl.all(), expr]).head()

col("a").sin().alias("sinus of a")


Time,a,b,c,sinus of a
f64,f64,f64,f64,f64
0.0,1.0,2.0,3.0,0.841471
0.020408,1.020408,2.020408,3.020408,0.852322
0.040816,1.040816,2.040816,3.040816,0.862817
0.061224,1.061224,2.061224,3.061224,0.872953
0.081633,1.081633,2.081633,3.081633,0.882726


In [15]:
# Derivative
dadt = polang("(shift(a, 1) - shift(a, -1)) / (shift(Time, 1) - shift(Time, -1))")
print(expr)

df = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2)})
df.select([pl.all(), dadt.alias("dadt")]).head()

[([(SHIFT col("a") by 1) - (SHIFT col("a") by -1)]) / ([(SHIFT col("Time") by 1) - (SHIFT col("Time") by -1)])]


Time,a,dadt
f64,f64,f64
0.0,1.0,null
0.020408,1.020408,1.0
0.040816,1.040816,1.0
0.061224,1.061224,1.0
0.081633,1.081633,1.0


In [39]:
from functools import reduce
from typing import Callable, List
from pandas import DataFrame
from polars import Expr, LazyFrame

FrameFunc = Callable[[LazyFrame | DataFrame], LazyFrame | DataFrame]

def to_function(expr: Expr) -> FrameFunc:
    return methodcaller("select", [expr])

def to_expansion(expr: Expr) -> List[FrameFunc]:
    return methodcaller("select", [pl.all(), expr])

def compose(f: Expr, g: Expr) -> FrameFunc:
    return lambda df: df.select(f).select(g)

def dfmap(fn: List[FrameFunc], df: LazyFrame) -> LazyFrame:
    return reduce(lambda f, g: g(f), fn, df)

In [41]:
dadt = polang("(shift(_a, 1) - shift(_a, -1)) / (shift(Time, 1) - shift(Time, -1))")
sina = polang("alias(sin(a), '_a')")

fn = [to_expansion(dadt), to_expansion(sina)][::-1]
lf = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2)}).lazy()

dfmap(fn, lf).collect()

DuplicateError: Column with name: '_a' has more than one occurrences

In [32]:
fn = [lambda x: x+1 for x in range(10)]
reduce(lambda f, g: g(f), fn, 0)

10